In [ ]:
import numpy as np 
import sys
import subprocess
import os
import matplotlib.pyplot as plt


In [ ]:
precision_list = [ x for x in range(3,53)]
print(precision_list)

number_of_elems_base = 10000 #we will multiply this at each step and produce based on that too 
nb_test = 10

In [ ]:
#compile normally 
original = os.system("clang++-7 reactor_simulator.cpp -o reactor_simulator_double -lm")
#compile with verificarlo
verif_compiled = os.system("verificarlo-c++ reactor_simulator.cpp -o reactor_simulator_verificarlo -lm")

In [ ]:
#execution with a backend phase
library_name= "libinterflop_vprec.so"
for elm in range (1, 1+nb_test):
    nb_elm = number_of_elems_base * elm
    for t in precision_list:     
        system_call = f"VFC_BACKENDS=\"{library_name} --precision-binary64={t}\"  ./reactor_simulator_verificarlo 1 1 0 {nb_elm}" 
        output_file_name=f"value_prec{t}_elm{nb_elm}.txt"
        with open(output_file_name, "w") as file:
            result = subprocess.run(system_call, shell=True, stdout=file,stderr=subprocess.PIPE)

        if result.returncode != 0:
            print(f"Error executing system call: {result.stderr.decode('utf-8')}")
        else:
            print(f"System call executed successfully, output written to {file}")


In [ ]:
def get_last_line(source_file):
    with open(source_file, 'r') as file:
        lines = file.readlines()
        if lines:
            return lines[-1]
        else:
            return None
        
def append_to_file(destination_file, line):
    with open(destination_file, 'a') as file:
        file.write(line+'\n')


def capture_last_stdout_line(command):
    result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    
    stdout_lines = result.stdout.strip().split('\n')
    if stdout_lines:
        return stdout_lines[-1]
    else:
        return None

In [ ]:
#aggregate the results per precision 

for elm in range (1,1+nb_test):
    nb_elm = number_of_elems_base * elm
    destination_file=f"value_elm{nb_elm}_agg.txt"
    for i in range(3,53):
        source_file=f"value_prec{i}_elm{nb_elm}.txt"
        last_line = get_last_line(source_file)
        if last_line is not None:
            append_to_file(destination_file, last_line)
            print(f"Last line from {source_file} appended to {destination_file}.")
        else:
            print(f"No lines found in {source_file}.")

In [ ]:
#execute the original double precision and get values
for it in range (1,nb_test+1):
    nb_elm = it * number_of_elems_base
    program = f"./reactor_simulator_double 1 1 0 {nb_elm}"
    output_name = f"double_prec_elm{nb_elm}.txt"

    last_line = capture_last_stdout_line(program)
    if last_line is not None:
        append_to_file(output_name, last_line)
        print(f"Last line of stdout from command '{program}' appended to {output_name}.")
    else:
        print(f"No output captured from command '{program}'.")

   

In [ ]:
#to make the difference between the double comp value and the emulated one 
x_array = np.arange(3,53)
x_indices = range(len(x_array))
for elm in range (1, nb_test +1):
    nb_elm = elm * number_of_elems_base
    filename1 = f"double_prec_elm{nb_elm}.txt"
    filename2 = f"value_elm{nb_elm}_agg.txt"
    orig_value = np.loadtxt(filename1, dtype=np.float64)
    verif_array = np.loadtxt(filename2, dtype=np.float64)
    diff = np.abs(orig_value - verif_array) 

    plt.figure(figsize=(15, 10))
    plt.plot(x_array, diff, marker='o', label=f"nb_elem = {nb_elm}")
    plt.axvline(x=23, color='r', linestyle='--', linewidth=2)
    plt.xticks(x_array)
    plt.xlabel ("Precision t")
    plt.ylabel("Error")
    plt.yscale('log')
    plt.title(f"Acummulated Error over precision on Total Energy variable")
    plt.grid(True)
    plt.legend()
    plt.show()


In [ ]:
#to make the difference between the double comp value and the emulated one 
x_array = np.arange(3,53)
x_indices = range(len(x_array))
plt.figure(figsize=(15, 10))
for elm in range (1, nb_test +1):
    nb_elm = elm * number_of_elems_base
    filename1 = f"double_prec_elm{nb_elm}.txt"
    filename2 = f"value_elm{nb_elm}_agg.txt"
    orig_value = np.loadtxt(filename1, dtype=np.float64)
    verif_array = np.loadtxt(filename2, dtype=np.float64)
    diff = np.abs(orig_value - verif_array) 

   
    plt.plot(x_array, diff, marker='o', linestyle ='none', label=f"nb_elem = {nb_elm}")
plt.axvline(x=23, color='r', linestyle='--', linewidth=2)

plt.xticks(x_array)
plt.xlabel ("Precision t")
plt.ylabel("Error")
plt.yscale('log')
plt.title(f"Acummulated Error over precision on Total Energy variable")
plt.grid(True)
plt.legend()
plt.show()